<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/preprocessing/SolarRadiationCSV_to_DF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hier werden die Solarstrahlungsdaten aus der Vorhersage von CAMS in einen Dataframe gespeichert:

**Dateiname: SolarRadiationCSV_to_DF**

Die Daten können als CSV-Datei über https://www.soda-pro.com/ heruntergeladen werden.

## Import der benötigten Module und allgemeines Setup:

In [ ]:
# Import der erfordelichen Module
import os
import numpy as np
import pandas as pd

In [ ]:
# Verbinden mit der Google-Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Pfad zur CSV-Datei auf Google Drive
import_folder = '/content/drive/My Drive/Colab_Notebooks/CSV_SolarRadiation/'
export_folder = '/content/drive/My Drive/Colab_Notebooks/CSV_SolarRadiation/'

import_file_name = 'CAMS_solar_radiation.csv'
export_file_name = 'solar_radiation_forecast.csv'

## Import der Vorhersagewerte:

In [ ]:
df_import = pd.read_csv(import_folder + import_file_name , sep = ';')
# Anpassen des Zeitstempels
df_import['Observation_period'] = pd.to_datetime(df_import['Observation_period'].str.slice(0, 19).str.replace('T', ' '))

## Aggregieren der Vorhersagewerte:

Die Daten liegen mit einer minütlichen Auflösung vor und werden im Folgenden zu 2 minütlichen Werten aggregiert.

In [ ]:
def zeilen_aggregieren(df):
    # Spalte erstellen, die zwei Zeilen eine Gruppe zuordnet
    df['group'] = (df.index // 2)

    # Gruppieren des Dataframes und anwenden der Aggregationsfunktion
    aggregationsfunktionen = {'Observation_period': 'first',  # Die erste Spalte bleibt unverändert
                             'Clear_sky_GHI': 'sum',          # Summe aller anderen Spalten
                             'Clear_sky_BHI': 'sum',
                             'GHI': 'sum',
                             'BHI': 'sum',
                             }

    zusammengefasstes_df = df.groupby('group').agg(aggregationsfunktionen).reset_index(drop=True)
    # Verschieben des Zeitstempels um eine Minute
    zusammengefasstes_df['Observation_period'] = zusammengefasstes_df['Observation_period'] + pd.to_timedelta(1, unit='m')

    return zusammengefasstes_df

In [ ]:
df_2min = zeilen_aggregieren(df_import)

0              0
1              0
2              1
3              1
4              2
           ...  
658075    329037
658076    329038
658077    329038
658078    329039
658079    329039
Name: group, Length: 658080, dtype: int64


## Abspeichern der Globalstrahlungs-Vorhersagewerte in einen Dataframe:

In [ ]:
def save_dataframe_to_drive(dataframe, folder_path, filename):

    # Erstelle den vollen Pfad zur Datei
    full_path = os.path.join(folder_path, filename)

    # Speichere den DataFrame als CSV-Datei auf Google Drive
    dataframe.to_csv(full_path, index=False)

    print(f'Der DataFrame wurde als {filename} in {folder_path} auf Google Drive gespeichert.')

In [ ]:
save_dataframe_to_drive(df_2min, export_folder, export_file_name)